## Tensorflow Object Detection API 简单测试

In [1]:
# 载入套件
import os
import pathlib
import tensorflow as tf
import pathlib

## GPU 记忆体配置设定

In [2]:
# GPU 设定为 记忆体动态调整 (dynamic memory allocation)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
 GPU 设定为固定为 2GB 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:  # 1024*2：2048MB = 2GB
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*2)])

SyntaxError: invalid syntax (<ipython-input-3-618820cbc86d>, line 1)

## 下载模型

In [ ]:
# 下载模型，并解压缩
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    # 解压缩
    model_dir = tf.keras.utils.get_file(fname=model_name,
                            origin=base_url + model_date + '/' + model_file,
                            untar=True)
    return str(model_dir)

MODEL_DATE = '20200711'
MODEL_NAME = 'centernet_hg104_1024x1024_coco17_tpu-32'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)
PATH_TO_MODEL_DIR

## 检查模型目录

In [ ]:
# 读取 PATH_TO_MODEL_DIR 目录下所有目录及档案
from os import listdir
from os.path import isfile, join
from os import path

for f in listdir(PATH_TO_MODEL_DIR):
    print(f)

## 从下载的目录载入模型

In [ ]:
# 不显示警告讯息
import warnings
warnings.filterwarnings('ignore')   # Suppress warnings

In [ ]:
# 从下载的目录载入模型，耗时甚久
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('载入模型...', end='')
start_time = time.time()

# 载入模型
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print(f'共花费 {elapsed_time} 秒.')

## 建立 Label 的对照表

In [ ]:
# 下载 labels file
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models'
    base_url += '/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)
PATH_TO_LABELS

In [ ]:
# 建立 Label 的对照表 (代码与名称)
category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS, use_display_name=True)

## 选一张图片进行物件侦测

In [ ]:
# 任选一张图片进行物件侦测
import numpy as np
from PIL import Image

# 开启一张图片
image_np = np.array(Image.open('./images_Object_Detection/zebra.jpg'))

# 转为 TensorFlow tensor 资料型态
input_tensor = tf.convert_to_tensor(image_np)
# 加一维，变为 (笔数, 宽, 高, 颜色)
input_tensor = input_tensor[tf.newaxis, ...]

# detections：物件资讯 内含 (候选框, 类别, 机率)
detections = detect_fn(input_tensor)
num_detections = int(detections.pop('num_detections'))
print(f'物件个数：{num_detections}')
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}

detections['num_detections'] = num_detections
# 转为整数
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

print(f'物件资讯 (候选框, 类别, 机率)：')
for detection_boxes, detection_classes, detection_scores in \
    zip(detections['detection_boxes'], detections['detection_classes'], 
        detections['detection_scores']):
    print(np.around(detection_boxes,4), detection_classes, 
          round(detection_scores*100, 2))

## 筛选Bounding Box，并将图片的物件加框

In [ ]:
import matplotlib.pyplot as plt

image_np_with_detections = image_np.copy()
# 加框
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False)

# 显示，无效
plt.figure(figsize=(12,8))
plt.imshow(image_np_with_detections, cmap='viridis')
plt.show()

## 显示处理后的图片

In [ ]:
# 存档
saved_file = './images_Object_Detection/zebra._detection1.png'
plt.savefig(saved_file)

# 显示
from IPython.display import Image
Image(saved_file)

## 从下载的目录载入模型另一种方法，非常快速

In [1]:
# 快速从下载的目录载入模型
import time
from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# 组态档及模型档路径
PATH_TO_CFG = PATH_TO_MODEL_DIR + "/pipeline.config"
PATH_TO_CKPT = PATH_TO_MODEL_DIR + "/checkpoint"

# 计时开始
print('Loading model... ', end='')
start_time = time.time()

# 载入组态档，再建置模型
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, 
                                      is_training=False)

# 还原模型
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

# 计时完成
end_time = time.time()
elapsed_time = end_time - start_time
print(f'共花费 {elapsed_time} 秒.')

ModuleNotFoundError: No module named 'object_detection'

In [ ]:
# 任选一张图片进行物件侦测
import numpy as np 
from PIL import Image

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

# 读取图档
image_np = np.array(Image.open('./images_Object_Detection/zebra.jpg'))
input_tensor = tf.convert_to_tensor(image_np, dtype=tf.float32)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detect_fn(input_tensor)
num_detections = int(detections.pop('num_detections'))

print(f'物件个数：{num_detections}')
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
print(f'物件资讯 (候选框, 类别, 机率)：')
for detection_boxes, detection_classes, detection_scores in \
    zip(detections['detection_boxes'], detections['detection_classes'], 
        detections['detection_scores']):
    print(np.around(detection_boxes,4), int(detection_classes)+1, 
          round(detection_scores*100, 2))

# 结果存入 detections 变数
detections['num_detections'] = num_detections
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

## 候选框筛选，并将图片的物件加框

In [ ]:
# 将物件框起来
# min_score_thresh=.30 表机率(Confidence)至少要大于 30%
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes']+1,
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False)

plt.figure(figsize=(12,8))
plt.imshow(image_np_with_detections, cmap='viridis')
plt.show()

## 显示处理后的图片

In [ ]:
# 存档
saved_file = './images_Object_Detection/zebra._detection2.png'
plt.savefig(saved_file)

# 显示
from IPython.display import Image
Image(saved_file)